In [16]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 18, Finished, Available, Finished)

we used spark sql to insert data till now for order priority table we use pyspark to create a delta table as well as to implement incremental load

In [17]:
DeltaTable.createIfNotExists(spark)\
            .tableName("Gold_OrderPriority")\
            .addColumn("OrderPriority_ID",LongType())\
            .addColumn("Order_Priority",StringType())\
            .addColumn("Created_TS",TimestampType())\
            .addColumn("Modified_TS",TimestampType())\
            .execute()

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 19, Finished, Available, Finished)

Now we need to implement the incremental logic for that we need gold priority table into data frame

In [18]:
df = spark.read.table("Gold_OrderPriority")

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 20, Finished, Available, Finished)

In [19]:
Max_Date = df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 21, Finished, Available, Finished)

In [20]:
Max_Date

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 22, Finished, Available, Finished)

'2025-11-02 09:01:11.045267'

In [21]:
df_bronze = spark.read.table("Bronze_Sales")

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 23, Finished, Available, Finished)

In [22]:
df_bronze_mod = df_bronze.select("Order_Priority").where(col("Modified_TS")>Max_Date).drop_duplicates() #select() simpmply takes teh column, selectExpr() allows sql expression inside string eg. "upper(country)"

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 24, Finished, Available, Finished)

In [23]:
df_bronze_mod.show()

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 25, Finished, Available, Finished)

+--------------+
|Order_Priority|
+--------------+
+--------------+



Now we will add incremental column for that we need to use monotonically increasing ID function, we're creating this because we have OrderPriority_ID which is not inside our bronze table and this will be a key for our OrderPriority table. Before that we need to pick the maximum value of OrderPriority_ID becz if we have 10 records OrderPriority_ID goes from 1 to 10 again during next loading number will goes from 1 to 10 in order to avoid this we catch max value and start to add from that

In [24]:
Max_ID = df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

df_final = df_bronze_mod.withColumn("OrderPriority_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 26, Finished, Available, Finished)

In [25]:
df_final.show()

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 27, Finished, Available, Finished)

+--------------+----------------+
|Order_Priority|OrderPriority_ID|
+--------------+----------------+
+--------------+----------------+



Now we will write merge to insert data into our orderpriority table, so before doing this we need to take our orderpriority table as delta table and take inside a variable

In [26]:
df_gold_delta = DeltaTable.forName(spark,"Gold_OrderPriority")
df_bronze_table = df_final

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 28, Finished, Available, Finished)

In [27]:
df_gold_delta.alias("gold")\
                .merge(\
                df_bronze_table.alias("bronze"),\
                "Gold.Order_Priority==Bronze.Order_Priority"\
                )\
                .whenMatchedUpdate(\
                    set={
                        "Gold.Modified_TS":current_timestamp()
                    }
                )\
                .whenNotMatchedInsert(\
                    values={
                        "gold.OrderPriority_ID":"bronze.OrderPriority_ID",
                        "gold.Order_Priority": "bronze.Order_Priority",
                        "gold.Created_TS":current_timestamp(),
                        "gold.Modified_TS":current_timestamp()
                    }
                )\
                .execute()

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 29, Finished, Available, Finished)

In [28]:
%%sql
select * from gold_orderpriority

StatementMeta(, e74b8bcd-8738-4be2-a270-a28f8a4ef0d1, 30, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>